In [ ]:
import numpy as np
import pandas as pd

# mbpp = pd.read_csv('/kaggle/input/testing-data/python coding task - mbpp.csv')

# mbpp.drop(columns=['test_list'], inplace=True)
# mbpp = mbpp.sample(5)

# mbpp.head()

In [ ]:
# trial = pd.read_csv('/kaggle/input/testing-data/trial.csv')
# trial = trial.sample(5)

# trial.drop(columns=['test_list', 'id'], inplace=True)

# trial.head()

In [ ]:
nvidia = pd.read_csv('/kaggle/input/nvidia-test-data-update1/GoodCSV2 - GoodCSV2.csv.csv')
# nvidia = nvidia.sample(5)

nvidia.drop(columns=['test_list', 'id'], inplace=True)

nvidia.head()

In [ ]:
# df = pd.concat([trail, mbpp], ignore_index=True)

In [ ]:
# df = mbpp.sample(5)
# df.shape

In [ ]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# # Load NLLB translator model
# model_id = "facebook/nllb-200-distilled-600M"
# trans_tokenizer = AutoTokenizer.from_pretrained(model_id)
# trans_model = AutoModelForSeq2SeqLM.from_pretrained(model_id).to("cuda")

In [ ]:
# import pandas as pd
# import torch
# from tqdm.auto import tqdm

# # Step 1: Define the separator-based splitting function
# def split_instruction_and_example(text):
#     for sep in ["Example:", "উদাহরণ:", "Exammple:", " উদাহরণ:", " উদাহরণঃ", "Ex:"]:
#         if sep in text:
#             parts = text.split(sep, 1)
#             instruction = parts[0].strip()
#             example = parts[1].strip()
#             return instruction, example
#     return text.strip(), ""  # no example found

# # Step 2: Translation function (using your defined model & tokenizer)
# def translate_bn_to_en(text: str) -> str:
#     inputs = trans_tokenizer(
#         text,
#         return_tensors="pt",
#         padding=True,
#         truncation=True,
#         max_length=512
#     ).to("cuda")

#     translated_ids = trans_model.generate(
#         **inputs,
#         forced_bos_token_id=trans_tokenizer.convert_tokens_to_ids("eng_Latn"),
#         max_new_tokens=512
#     )

#     return trans_tokenizer.batch_decode(translated_ids, skip_special_tokens=True)[0]

# # Step 3: Process the dev.csv
# trial = pd.read_csv('/kaggle/input/testing-data/trial.csv')
# trial.drop(columns=['test_list', 'id'], inplace=True)

# df = trial.sample(5)

# assert {"instruction", "response"}.issubset(df.columns)

# # Step 4: Process each instruction (split + translate)
# final_prompts = []

# for raw in tqdm(df["instruction"], desc="🔁 Translating instructions"):
#     bangla_instr, example = split_instruction_and_example(raw)
#     # print(bangla_instr, example)

#     try:
#         english_instr = translate_bn_to_en(bangla_instr)
#     except Exception as e:
#         english_instr = "[Translation Failed]"
#         print("⚠️ Error:", e)

#     # Combine again (optional: add separator for clarity)
#     if example:
#         full_prompt = f"{english_instr.strip()} Example:\n{example.strip()}"
#     else:
#         full_prompt = english_instr.strip()

#     # print(full_prompt)
#     final_prompts.append(full_prompt)
#     # break

# # for ls in final_prompts:
# #     print(ls)

# # Step 5: Create a new DataFrame with id and translated_prompt
# trial_df = pd.DataFrame({
#     "instruction": final_prompts,
#     "response": df['response']
# })


# # Save to CSV in /kaggle/working
# trial_df.to_csv("/kaggle/working/trial_translated.csv", index=False)
# print("✅ Saved translated file → /kaggle/working/dev_translated.csv")

In [ ]:
# df = pd.concat([trial, mbpp, nvidia], ignore_index=True)

In [ ]:
df = nvidia

In [ ]:
df.shape

In [ ]:
df.head(5)

## Step 0 - Installing Librariess

In [ ]:
!pip install -q --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
!pip install -q sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
!pip install -q --no-deps unsloth

In [ ]:
guideline = (
    "You are an ULTIMATE Python Coding Expert. Follow the rules STRICTLY.\n"
    "Input: A programming instruction in English (may include a function name and parameters).\n"
    "Output: Only the correct Python code inside a fenced code block.\n\n"
    "Format:\n"
    "```python\n"
    "<code here>\n"
    "```\n\n"
    "Rules:\n"
    "- Reason internally; output only the final code block.\n"
    "- Handle edge cases; the code MUST pass typical unit tests.\n"
    "- Preserve the exact function name and parameters if provided.\n"
    "- Always return values (no print() / input()).\n"
    "- Do not define classes; use functions and variables only.\n"
    "- Import required libraries if needed (no unused imports).\n"
    "- Do not include any text, explanations, comments, or docstrings.\n"
    "\n"
)


In [ ]:
import pandas as pd
import re

# === Load your raw dataset ===

# === Keep only the needed columns ===
keep_cols = ["instruction", "response"]
df = df[[c for c in keep_cols if c in df.columns]].copy()

# === Basic cleaning ===
# 1) Drop missing / empty
df.dropna(subset=["instruction", "response"], inplace=True)
df["instruction"] = df["instruction"].astype(str).str.strip()
df["response"] = df["response"].astype(str).str.strip()
df = df[(df["instruction"] != "") & (df["response"] != "")]

# 2) Remove any accidental test-leakage columns if present
# (Just in case your source has test_list or reference outputs)
for col in list(df.columns):
    if col not in keep_cols:
        df.drop(columns=[col], inplace=True)

# 3) Ensure response is ONLY a fenced Python code block
def to_fenced_python(s: str) -> str:
    s = s.strip()
    # If it's already fenced python, keep; otherwise wrap it.
    fenced = re.match(r"^```python\s[\s\S]*?\n```$", s.strip())
    if fenced:
        return s
    # Strip any stray markdown and wrap
    s = re.sub(r"^```.+?\n", "", s)         # remove any starting fence
    s = re.sub(r"\n```$", "", s)            # remove trailing fence
    return f"```python\n{s}\n```"

df["response"] = df["response"].apply(to_fenced_python)

# 4) Length filters (optional but helpful)
# Remove super-short or ultra-long pairs that hurt training stability
df = df[(df["instruction"].str.len() >= 8) & (df["response"].str.len() >= 16)]
df = df[df["response"].str.len() <= 12000]  # ~ reasonable upper bound

# 5) Deduplicate near-identical pairs
df["__key__"] = (
    df["instruction"].str.lower().str.replace(r"\s+", " ", regex=True).str.strip()
    + "||"
    + df["response"].str.lower().str.replace(r"\s+", " ", regex=True).str.strip()
)
df = df.drop_duplicates(subset="__key__").drop(columns="__key__")

# 6) Save cleaned data for next steps
df.to_json("/kaggle/working/train_clean.jsonl", orient="records", lines=True, force_ascii=False)
print(f"✅ Cleaned pairs: {len(df)}")

In [ ]:
df.columns

In [ ]:
import torch, sys
print(torch.__version__)
print(sys.modules.get("torch"))


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load NLLB translator model
model_id = "facebook/nllb-200-distilled-600M"
trans_tokenizer = AutoTokenizer.from_pretrained(model_id)
trans_model = AutoModelForSeq2SeqLM.from_pretrained(model_id).to("cuda")

In [ ]:
df.columns

In [ ]:
import pandas as pd
import torch
from tqdm.auto import tqdm

# Step 1: Define the separator-based splitting function
def split_instruction_and_example(text):
    for sep in ["Example:", "উদাহরণ:", "Exammple:", " উদাহরণ:", " উদাহরণঃ", "Ex:"]:
        if sep in text:
            parts = text.split(sep, 1)
            instruction = parts[0].strip()
            example = parts[1].strip()
            return instruction, example
    return text.strip(), ""  # no example found

# Step 2: Translation function (using your defined model & tokenizer)
def translate_bn_to_en(text: str) -> str:
    inputs = trans_tokenizer(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to("cuda")

    translated_ids = trans_model.generate(
        **inputs,
        forced_bos_token_id=trans_tokenizer.convert_tokens_to_ids("eng_Latn"),
        max_new_tokens=512
    )

    return trans_tokenizer.batch_decode(translated_ids, skip_special_tokens=True)[0]

# Step 3: Process the dev.csv
# df = pd.read_csv("/kaggle/input/blptask2data/dev_v2.csv")

assert {"instruction", "response"}.issubset(df.columns)

# Step 4: Process each instruction (split + translate)
final_prompts = []

for raw in tqdm(df["instruction"], desc="🔁 Translating instructions"):
    bangla_instr, example = split_instruction_and_example(raw)
    # print(bangla_instr, example)

    try:
        english_instr = translate_bn_to_en(bangla_instr)
    except Exception as e:
        english_instr = "[Translation Failed]"
        print("⚠️ Error:", e)

    # Combine again (optional: add separator for clarity)
    if example:
        full_prompt = f"{english_instr.strip()} Example:\n{example.strip()}"
    else:
        full_prompt = english_instr.strip()

    # print(full_prompt)
    final_prompts.append(full_prompt)
    # break

# for ls in final_prompts:
#     print(ls)

# Step 5: Create a new DataFrame with id and translated_prompt
translated_df = pd.DataFrame({
    "instruction": final_prompts,
    "response": df['response']
})


# Save to CSV in /kaggle/working
translated_df.to_csv("/kaggle/working/dev_translated.csv", index=False)
print("✅ Saved translated file → /kaggle/working/dev_translated.csv")

In [ ]:
# STEP 5 — Apply Qwen chat template (system, user, assistant)
import json
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer

# 0) Config
QWEN_MODEL_ID = "Qwen/Qwen2.5-Coder-3B-Instruct"  # or your chosen instruct model
INPUT_PATH = "/kaggle/working/dev_translated.csv"  # from step 4
OUTPUT_PATH = "/kaggle/working/train_chat_templated.jsonl"


# 2) Load tokenizer (only to render chat template)
tokenizer = AutoTokenizer.from_pretrained(QWEN_MODEL_ID, use_fast=True)

# 3) Load cleaned EN-instruction dataset (instruction|response)
df = pd.read_csv(INPUT_PATH)

# 4) Build messages → render to chat-formatted text
#    We include assistant content in the rendered text, so set add_generation_prompt=False
def render_row(inst: str, resp: str) -> str:
    messages = [
        {"role": "system", "content": guideline},
        {"role": "user", "content": inst},
        {"role": "assistant", "content": resp},
    ]
    return tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )

# 5) Stream to jsonl (avoid huge in-memory list)
with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
    for _, row in tqdm(df.iterrows(), total=len(df)):
        text = render_row(row["instruction"], row["response"])
        f.write(json.dumps({"text": text}, ensure_ascii=False) + "\n")

print("✅ Chat template applied →", OUTPUT_PATH)

In [ ]:
pd.set_option("display.max_colwidth", None)

In [ ]:
dff = pd.read_json(OUTPUT_PATH, lines=True)
dff.head(2)

In [ ]:
# STEP 6 — Tokenize (and optionally pack)
from datasets import load_dataset
from transformers import AutoTokenizer
import torch

QWEN_MODEL_ID = "Qwen/Qwen2.5-Coder-3B-Instruct"   # keep consistent with Step 5
INPUT_PATH = "/kaggle/working/train_chat_templated.jsonl"
MAX_SEQ_LEN = 1024
PACK = False  # set True later if you want packed LM training

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(QWEN_MODEL_ID, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load chat-formatted jsonl -> HF dataset
ds = load_dataset("json", data_files=INPUT_PATH, split="train")

# Tokenize fn
def tok_fn(batch):
    out = tokenizer(
        batch["text"],
        max_length=MAX_SEQ_LEN,
        truncation=True,
        padding=False,               # use dynamic padding in collator later
        return_attention_mask=True,
    )
    # For causal LM, labels = input_ids
    out["labels"] = out["input_ids"].copy()
    return out

ds_tok = ds.map(tok_fn, batched=True, remove_columns=["text"])
print(ds_tok)

# (Optional) Packing — disabled by default
if PACK:
    block_size = MAX_SEQ_LEN
    def group_texts(examples):
        # Flatten
        concatenated = {k: sum(examples[k], []) for k in ["input_ids", "attention_mask", "labels"]}
        total_length = len(concatenated["input_ids"])
        total_length = (total_length // block_size) * block_size
        result = {
            k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
            for k, t in concatenated.items()
        }
        return result
    ds_tok = ds_tok.map(group_texts, batched=True)
    print("✅ Packed dataset:", ds_tok)

# Save tokenized dataset for reuse
ds_tok.save_to_disk("/kaggle/working/ds_tokenized")
print("✅ Saved tokenized dataset → /kaggle/working/ds_tokenized")


In [ ]:
# STEP 7 — Use the whole dataset for training (no val split)
# (Nothing to do beyond naming it explicitly)

train_dataset = ds_tok
print("✅ Training samples:", len(train_dataset))


In [ ]:
# STEP 8 — Load model in 4-bit (QLoRA-ready)
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# QWEN_MODEL_ID = "Qwen/Qwen2.5-Coder-3B-Instruct"

# Compute dtype: use bf16 if supported, else fp16 (T4 will use fp16)
compute_dtype = torch.bfloat16 if (torch.cuda.is_available() and torch.cuda.is_bf16_supported()) else torch.float16

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

tokenizer = AutoTokenizer.from_pretrained(QWEN_MODEL_ID, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    QWEN_MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# Ensure pad id is set on model
model.config.pad_token_id = tokenizer.pad_token_id

print("✅ Model loaded in 4-bit with compute dtype:", compute_dtype)


In [ ]:
# STEP 9
model.gradient_checkpointing_enable()
model.config.use_cache = False
model.enable_input_require_grads()  # required for 4-bit + LoRA
print("✅ Gradient checkpointing ON; use_cache=False; inputs require grads.")

In [ ]:
# STEP 10
from peft import LoraConfig, TaskType

LORA_R = 48
LORA_ALPHA = 96
LORA_DROPOUT = 0.05
TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]

lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=TARGET_MODULES,
)
print("✅ LoRA config ready.")


In [ ]:
# STEP 11
from peft import get_peft_model

model = get_peft_model(model, lora_config)

# (Optional) sanity: show trainable vs total params
def print_trainable_parameters(m):
    trainable = 0
    total = 0
    for _, p in m.named_parameters():
        num = p.numel()
        total += num
        if p.requires_grad:
            trainable += num
    print(f"Trainable params: {trainable:,} | Total params: {total:,} | Trainable%: {100 * trainable/total:.4f}%")

print_trainable_parameters(model)
print("✅ PEFT adapters attached.")

In [ ]:
# STEP 12
from transformers import TrainingArguments

OUTPUT_DIR = "/kaggle/working/qwen-coder-qlora"

train_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    weight_decay=0.0,
    fp16=True,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",

    # 👇 these make Trainer print a loss line at the end of each epoch
    logging_strategy="epoch",
    save_strategy="epoch",

    report_to="none",
    logging_steps=50,          # ignored when logging_strategy="epoch"
    save_total_limit=2,
    dataloader_num_workers=2,
)
print("✅ TrainingArguments ready.")


In [ ]:
# STEP 13 — Initialize Trainer (dynamic padding; labels created by collator)

from datasets import load_from_disk
from transformers import DataCollatorForLanguageModeling, Trainer

# Reload tokenized dataset saved in Step 6
train_dataset = load_from_disk("/kaggle/working/ds_tokenized")

# Drop precomputed labels to avoid length mismatch during padding
if "labels" in train_dataset.column_names:
    train_dataset = train_dataset.remove_columns("labels")

# Ensure right-padding for causal LM
tokenizer.padding_side = "right"

# Collator: pads dynamically and sets labels = input_ids (mlm=False)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    pad_to_multiple_of=8,   # optional, speeds up on tensor cores
)

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=train_dataset,   # full dataset (no val)
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("✅ Trainer initialized (dynamic padding + labels).")


In [ ]:
# STEP 14 — Train
train_result = trainer.train()

# Save PEFT adapters and tokenizer
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print("✅ Training finished.")
print(train_result)

#############################

In [ ]:
# Reload (or reuse) tokenizer/model if needed
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch, re, json

QWEN_MODEL_ID = "Qwen/Qwen2.5-Coder-3B-Instruct"
OUTPUT_DIR     = "/kaggle/working/qwen-coder-qlora"  

compute_dtype = torch.bfloat16 if (torch.cuda.is_available() and torch.cuda.is_bf16_supported()) else torch.float16
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True,
                                bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=compute_dtype)

tokenizer = AutoTokenizer.from_pretrained(QWEN_MODEL_ID, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

base = AutoModelForCausalLM.from_pretrained(QWEN_MODEL_ID, quantization_config=bnb_config,
                                            device_map="auto", trust_remote_code=True)

# Attach trained LoRA adapters
from peft import PeftModel
model = PeftModel.from_pretrained(base, OUTPUT_DIR)
model.eval()


def render_prompt(instruction: str) -> str:
    msgs = [
        {"role": "system", "content": guideline},
        {"role": "user", "content": instruction}
    ]
    return tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)

CODE_BLOCK_RE = re.compile(r"```python\s*([\s\S]*?)```", re.IGNORECASE)

def extract_code(text: str) -> str:
    m = CODE_BLOCK_RE.search(text)
    return m.group(1).strip() if m else text.strip()

@torch.inference_mode()
def generate_code(instruction: str,
                  max_new_tokens=512, temperature=0.2, top_p=0.9, rep_penalty=1.05):
    text = render_prompt(instruction)
    inputs = tokenizer([text], return_tensors="pt").to(model.device)
    out_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        repetition_penalty=rep_penalty,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
    gen = tokenizer.decode(out_ids[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
    return extract_code(gen)

# Quick sanity check
print(generate_code("Write a function sum_series(n) that returns the sum of first n natural numbers."))


In [ ]:
from peft import PeftModel

print(type(model))                     # should show a PEFT wrapper class, not Qwen2ForCausalLM
print(getattr(model, "peft_config", None))  # should be a dict like {'default': LoraConfig(...)}

# DEVV

In [ ]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# # Load NLLB translator model
# model_id = "facebook/nllb-200-distilled-600M"
# trans_tokenizer = AutoTokenizer.from_pretrained(model_id)
# trans_model = AutoModelForSeq2SeqLM.from_pretrained(model_id).to("cuda")

In [ ]:
def translate_bn_to_en(text: str) -> str:
    inputs = trans_tokenizer(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to("cuda")

    # 🔄 Translate Bangla → English using known language code token
    translated_ids = trans_model.generate(
        **inputs,
        forced_bos_token_id=trans_tokenizer.convert_tokens_to_ids("eng_Latn"),  # ✅ fixed here
        max_new_tokens=256
    )

    translated_text = trans_tokenizer.batch_decode(translated_ids, skip_special_tokens=True)[0]
    return translated_text

In [ ]:
import pandas as pd
import torch
from tqdm.auto import tqdm

# Step 1: Define the separator-based splitting function
def split_instruction_and_example(text):
    for sep in ["Example:", "উদাহরণ:", "Exammple:", " উদাহরণ:", " উদাহরণঃ", "Ex:"]:
        if sep in text:
            parts = text.split(sep, 1)
            instruction = parts[0].strip()
            example = parts[1].strip()
            return instruction, example
    return text.strip(), ""  # no example found

# Step 2: Translation function (using your defined model & tokenizer)
def translate_bn_to_en(text: str) -> str:
    inputs = trans_tokenizer(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to("cuda")

    translated_ids = trans_model.generate(
        **inputs,
        forced_bos_token_id=trans_tokenizer.convert_tokens_to_ids("eng_Latn"),
        max_new_tokens=256
    )

    return trans_tokenizer.batch_decode(translated_ids, skip_special_tokens=True)[0]

# Step 3: Process the dev.csv
df = pd.read_csv("/kaggle/input/testing-data/dev_v2.csv")
assert {"id", "instruction"}.issubset(df.columns)

# Step 4: Process each instruction (split + translate)
final_prompts = []

for raw in tqdm(df["instruction"], desc="🔁 Translating instructions"):
    bangla_instr, example = split_instruction_and_example(raw)
    # print(bangla_instr, example)

    try:
        english_instr = translate_bn_to_en(bangla_instr)
    except Exception as e:
        english_instr = "[Translation Failed]"
        print("⚠️ Error:", e)

    # Combine again (optional: add separator for clarity)
    if example:
        full_prompt = f"{english_instr.strip()} Example:\n{example.strip()}"
    else:
        full_prompt = english_instr.strip()

    # print(full_prompt)
    final_prompts.append(full_prompt)
    # break

# for ls in final_prompts:
#     print(ls)

# Step 5: Create a new DataFrame with id and translated_prompt
translated_df = pd.DataFrame({
    "id": df["id"],
    "instruction": final_prompts
})

# Save to CSV
translated_df.to_csv("dev_translated.csv", index=False)
print("✅ Saved translated file → dev_translated.csv")

In [ ]:
translated_df.shape

In [ ]:
translated_df.head(3)

## Step 8 - Use New Model on Dev Set

In [ ]:
# === DEV INFERENCE: use fine-tuned Qwen (base + LoRA) ===
import pandas as pd
import json, re, zipfile, torch
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# ---- Config (make sure these match your trained run) ----
# QWEN_MODEL_ID = "Qwen/Qwen2.5-Coder-3B-Instruct"   # or 7B, exactly the same as training
OUTPUT_DIR     = "/kaggle/working/qwen-coder-qlora"  # where your adapters were saved
DEV_IN_PATH    = "dev_translated.csv"                # your translated file: id,instruction
PRED_JSON      = "dev_predictions.json"              # prediction dump
SUB_JSON       = "submission.json"                   # Kaggle-style (id, response)

# ---- Load model + tokenizer (4-bit) ----
compute_dtype = torch.bfloat16 if (torch.cuda.is_available() and torch.cuda.is_bf16_supported()) else torch.float16
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=compute_dtype,
)

tokenizer = AutoTokenizer.from_pretrained(QWEN_MODEL_ID, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

_base = AutoModelForCausalLM.from_pretrained(
    QWEN_MODEL_ID, quantization_config=bnb_config, device_map="auto", trust_remote_code=True
)
model = PeftModel.from_pretrained(_base, OUTPUT_DIR)
model.eval()


def render_prompt(instruction: str) -> str:
    msgs = [
        {"role": "system", "content": guideline},
        {"role": "user",  "content": instruction.strip()}
    ]
    return tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)

CODE_BLOCK_RE = re.compile(r"```python\s*([\s\S]*?)```", re.IGNORECASE)
def extract_code(text: str) -> str:
    m = CODE_BLOCK_RE.search(text)
    return m.group(1).strip() if m else text.strip()


# After loading model + adapters
gc = model.generation_config
gc.do_sample = False
gc.temperature = None
gc.top_p = None
gc.top_k = None
gc.num_beams = 1  # keep greedy

@torch.inference_mode()
def generate_code(instruction: str,
                  max_new_tokens=512, temperature=0.0, top_p=1.0, rep_penalty=1.05):
    # deterministic for eval: temperature=0.0, top_p=1.0, do_sample=False
    text = render_prompt(instruction)
    inputs = tokenizer([text], return_tensors="pt").to(model.device)
    out_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        repetition_penalty=rep_penalty,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
    gen = tokenizer.decode(out_ids[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
    return extract_code(gen)

# ---- Run inference over dev_translated.csv ----
df = pd.read_csv(DEV_IN_PATH)
assert {"id", "instruction"}.issubset(df.columns), "dev_translated.csv must have id,instruction"

pred_rows = []
for rid, instr in tqdm(zip(df["id"], df["instruction"]), total=len(df), desc="🔁 Generating"):
    code = generate_code(instr)
    fenced = f"```python\n{code}\n```"
    pred_rows.append({"id": int(rid), "instruction": instr, "response": fenced})

# Save predictions (debug-friendly) and Kaggle submission
with open(PRED_JSON, "w", encoding="utf-8") as f:
    json.dump(pred_rows, f, ensure_ascii=False, indent=2)

sub_rows = [{"id": r["id"], "response": r["response"]} for r in pred_rows]
with open(SUB_JSON, "w", encoding="utf-8") as f:
    json.dump(sub_rows, f, ensure_ascii=False, indent=2)

print(f"✅ Wrote {PRED_JSON}, {SUB_JSON}")

## Step 9 - Preparing Submission File

In [ ]:
import json, os, re, zipfile

SUB_PATH = "submission.json"

def file_format_check(path: str) -> bool:
    # name + extension
    if os.path.basename(path) != "submission.json":
        print("Error: File name must be exactly 'submission.json'")
        return False
    if not path.lower().endswith(".json"):
        print("Error: File must have .json extension")
        return False

    # must be valid JSON (not JSONL) and root must be a list
    try:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
    except json.JSONDecodeError as e:
        print(f"Error: Invalid JSON format - {e}")
        print("Note: The file must be in proper JSON format (not JSONL)")
        return False

    if not isinstance(data, list):
        print("Error: The root element should be a list of objects")
        return False

    # each item: dict with ONLY keys {'id','response'}; id=int; response=str
    for idx, item in enumerate(data):
        if not isinstance(item, dict):
            print(f"Error: Item at index {idx} is not a dictionary")
            return False
        keys = set(item.keys())
        if keys != {"id", "response"}:
            print(f"Error: Item at index {idx} must contain only keys 'id' and 'response', found: {keys}")
            return False
        if not isinstance(item["id"], int):
            print(f"Error: 'id' field at index {idx} must be an integer")
            return False
        if not isinstance(item["response"], str):
            print(f"Error: 'response' field at index {idx} must be a string")
            return False

    print("Format check passed successfully!")
    return True

# ---------- Load, compute per-item validity, blank invalids, save, zip ----------
# Load JSON list
with open(SUB_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

n = len(data)
fence_pat = re.compile(r"^```python[\s\S]*```$", re.MULTILINE)

valid_format = []
valid_fence  = []
valid_both   = []

# Per-item validation mirrors file checker semantics
def item_format_ok(item):
    return (
        isinstance(item, dict)
        and set(item.keys()) == {"id", "response"}
        and isinstance(item["id"], int)
        and isinstance(item["response"], str)
    )

for item in data:
    vfmt = item_format_ok(item)
    vf   = bool(fence_pat.match(item["response"])) if vfmt else False
    valid_format.append(vfmt)
    valid_fence.append(vf)
    valid_both.append(vfmt and vf)

# After computing valid_format, valid_fence, valid_both
for i, item in enumerate(data):
    if not valid_format[i]:
        print(f"❌ Format Error at index {i}: {item}")
    elif not valid_fence[i]:
        print(f"❌ Fencing Error at index {i} (id={item.get('id')}):")
        print(item["response"])
        print("-" * 50)


# Report stats
nf = sum(valid_fence)
nm = sum(valid_format)
nb = sum(valid_both)
den = max(n, 1)
print(f"Fencing valid: {nf}/{n} ({nf*100.0/den:.1f}%)")
print(f"Format valid:  {nm}/{n} ({nm*100.0/den:.1f}%)")
print(f"Both valid:    {nb}/{n} ({nb*100.0/den:.1f}%)")

# Strict policy: blank responses that fail ANY check
for i, ok in enumerate(valid_both):
    if not ok and isinstance(data[i], dict) and "response" in data[i]:
        data[i]["response"] = ""

# Overwrite submission.json (id+response only)
with open(SUB_PATH, "w", encoding="utf-8") as f:
    json.dump(
        [{"id": item["id"], "response": item["response"]} for item in data],
        f, ensure_ascii=False, indent=2
    )
print("✅ Updated submission.json after checks (invalid responses blanked).")

# Final file-level check (should pass)
_ = file_format_check(SUB_PATH)

# Zip as submission.zip (Jupyter-friendly, no shell commands)
with zipfile.ZipFile("submission.zip", "w", compression=zipfile.ZIP_DEFLATED) as zf:
    zf.write(SUB_PATH)
print("📦 Created submission.zip containing submission.json.")


In [ ]:
# with open("/kaggle/working/submission.json", "r", encoding="utf-8") as f:
#     content = f.read()

# print(content)

# Submit the submission.zip file in CodaBench

### Save the NEW model...if it's good :)

In [ ]:
# model.save_pretrained("New_Model")  # Local saving
# tokenizer.save_pretrained("New_Model")